# Artigo 4 - Identificador de Spam


## Autor
Nome: Danilo César Tertuliano Melo

Matricula: 221031149

Github: DaniloCTM

----

## Objetivo
Este artigo aplica conceitos e técnicas de processamento de linguagem natural para identificar se as mensagens são spam ou não.

----

# Passo 1 - Configurando o Ambiente

Nessa etapa instalamos as bibliotecas pandas e numpy, e definimos os principais caminhos para os arquivos com os dados de treino e teste.

In [1]:
import pandas as pd
import numpy as np

In [2]:
base_path = '/kaggle/input/email-classification-nlp'
train_df = f'{base_path}/SMS_train.csv'
test_df = f'{base_path}/SMS_test.csv'

# Passo 2 - Organizando o Dataset

Utilizamos o pandas para ler os arquivos csv do dataset e definimos o encoding como latin-1 para remover alguns caracteres especiais

In [3]:
train_df = pd.read_csv(train_df,encoding='latin-1')

Aqui realizamos algumas mudanças no dataset, primeiro passamos os valores de "Non-Spam" para 0 e os de "Spam" para 1. Em seguida criamos a coluna de input com os dados da mensagem.

In [4]:
train_df['Label'] =  train_df['Label'].map({"Non-Spam":0.0,"Spam":1.0})
train_df['input'] = 'MESSAGE: ' + train_df.Message_body

In [5]:
train_df.head()

,S. No.,Message_body,Label,input
0,1,Rofl. Its true to its name,0.0,MESSAGE: Rofl. Its true to its name
1,2,The guy did some bitching but I acted like i'd...,0.0,MESSAGE: The guy did some bitching but I acted...
2,3,"Pity, * was in mood for that. So...any other s...",0.0,"MESSAGE: Pity, * was in mood for that. So...an..."
3,4,Will ü b going to esplanade fr home?,0.0,MESSAGE: Will ü b going to esplanade fr home?
4,5,This is the 2nd time we have tried 2 contact u...,1.0,MESSAGE: This is the 2nd time we have tried 2 ...


# Passo 3 - Fazendo a Tokenização

Inicialmente passamos o data frame para o formato de dataset do hugging face

In [6]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(train_df)

Aqui podemos ver algumas informações gerais sobre o dataset, como as colunas e o número de linhas.

In [7]:
ds

Dataset({
    features: ['S. No.', 'Message_body', 'Label', 'input'],
    num_rows: 957
})

Agora selecionamos o modelo, usar um modelo muito pesado pode causar problemas caso o seu ambiente não tenha recursos o suficiente. É possível encontrar modelos que foram treinados para problemas semelhantes ao seu na Hugging Face. Para essa aplicação foi utilizado o modelo deberta-v3-small.

In [8]:
model_nm = 'microsoft/deberta-v3-small'

É realizado o download do vocabulário e dos detalhes do modelo que escolhemos.

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Em seguida utlizamos uma linha do dataset para ver como funciona a tokenização. (É possível perceber que esse modelo separa as palavras por '_')

In [10]:
tokz.tokenize("Will ü b going to esplanade fr home?")

['▁Will',
 '▁ü',
 '▁b',
 '▁going',
 '▁to',
 '▁es',
 'plan',
 'ade',
 '▁fr',
 '▁home',
 '?']

Durante o processo de tokenização, a frase é dividida em tokens, que são unidades menores de texto, geralmente palavras, subpalavras ou caracteres. A tokenização é realizada com base no vocabulário do modelo em uso. Se uma palavra estiver presente no vocabulário, ela será representada por um único token. No entanto, se uma palavra não existir no vocabulário do modelo, ela será dividida em partes menores, para que possa ser representada.

Como mostrado no exemplo a palavra "esplande", não está presente no vocabulário do modelo, então ele divide ela em palavras menores. É assim que o modelo lida com palavras desconhecidas.

Agora realizamos a tokenização de todo o dataset de treino e é atribuido um index para cada palavra do vocabulário, assim podemos representar os tokens de forma numérica.

In [ ]:
#Função para realizar a tokenização da coluna input
def tok_func(x): return tokz(x["input"])

tok_ds = ds.map(tok_func, batched=True)

Visualizamos como os dados ficam.

In [12]:
row = tok_ds[0]
row['input'], row['input_ids']

('MESSAGE: Rofl. Its true to its name',
 [1, 81360, 294, 11288, 20516, 260, 2952, 980, 264, 359, 601, 2])

In [13]:
#Aparentemente os modelos do hugging face precisam que o campo do resultado se chame labels
tok_ds = tok_ds.rename_columns({'Label':'labels'})

----

# Passo 4 - Definindo a Métrica

Agora definimos a metrica que será utilizada para calcular o desempenho do modelo, nesse caso utilizamos a correlação de Pearson que foi apresentada pelo Jeremy durante a lição 4.

In [14]:
def corr(x,y): return np.corrcoef(x,y)[0][1]

In [15]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

-----------

# Passo 5 - dividindo os DataSets

Nessa etapa é importante entender a diferença entre os tipos de datasets:
* Dataset de treino: utilizado pelo modelo durante o processo de treinamento, é com ele que o modelo aprende.
* Dataset de validação: Esse dataset também é utilizado durante o treinamento, mas o modelo não aprende com ele, é usado apenas para ver como está o desempenho do modelo para dados diferentes.
* Dataset de teste: É utilizado quando o modelo já está pronto, esse dado serve para comprovar que o modelo está funcionando.

In [16]:
dds = tok_ds.train_test_split(0.20, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['S. No.', 'Message_body', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 765
    })
    test: Dataset({
        features: ['S. No.', 'Message_body', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 192
    })
})

-------------

# Passo 6 - Realizando o Treinamento

Começamos importando alguns elementos da biblioteca transformers do Hugging Face.

In [ ]:
from transformers import TrainingArguments,Trainer

Agora definimos algumas configurações para o treinamento:
*     batch size: Onde escolhemos o tamanho do batch para que os dados sejam processados de forma paralela (O uso de um valor muito alto pode resultar em um erro devido a falta de memória da gpu).
*     epochs: Aqui escolhemos a quantidade de épocas que o modelo será treinado.
*     learning rate: Aqui escolhemos a taxa de aprendizado do modelo.

In [18]:
bs = 32 #definindo o tamanho do batch
epochs = 4 #quantidade de épocas
lr = 8e-5 # taxa de aprendizado

Passamos os dados definidos acima para o TrainingArguments

In [19]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

Criamos o modelo e definimos o dataset de treino e o de teste. O Trainer funciona de forma semelhante ao Learner do fastai.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Realizamos o treinamento usando o método .train()

In [21]:
trainer.train();

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.033822,0.927494
2,No log,0.007051,0.966755
3,No log,0.011051,0.950428
4,No log,0.007810,0.963412


----
# Passo 7 - Configurando o dataset de teste

Repetimos a etapa de ler o csv, criar a coluna de input e fazer a tokenização dos dados.

In [23]:
test_df = pd.read_csv(test_df,encoding='latin-1')

In [24]:
test_df['input'] = 'MESSAGE: ' + test_df.Message_body

In [25]:
eval_ds = Dataset.from_pandas(test_df)

In [26]:
eval_tok_ds = eval_ds.map(tok_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

# Passo 8 - Fazendo a Predição

Fazemos a predição no dataset de teste. Nessa predição o modelo retorna o número 1 para Spam e 0 para Non-Spam.

In [33]:
preds = trainer.predict(eval_tok_ds).predictions.astype(int)
preds

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

Agora passamos os valores de volta para string e comparamos o valor obtido com os valores esperados.

In [34]:
preds_results = []
for i in preds:
    if i == 1:
        preds_results.append('Spam')
    else:
        preds_results.append('Non-Spam')

In [35]:
test_results = []
for i in test_df['Label']:
    test_results.append(i)

In [50]:
wrongs = 0 
for i in range(len(preds_results)):
    if preds_results[i] != test_results[i]:
        wrongs += 1
        print(f'{i}|{preds_results[i]} != {test_results[i]} ')
print(f'\nTaxa de erro: {(wrongs/125)*100}%')

32|Non-Spam != Spam 
34|Non-Spam != Spam 
51|Non-Spam != Spam 
55|Non-Spam != Spam 
57|Non-Spam != Spam 
64|Non-Spam != Spam 

Taxa de erro: 4.8%


# Deploy do Modelo

Na etapa final realizamos o login no huggingface e subimos o nosso modelo já treinado para realizar as predições.

In [ ]:
pip install --upgrade huggingface_hub

In [2]:
import huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.create_repo(repo_id="lesson-4")

In [ ]:
trainer.push_to_hub("lesson-4")

Para acessar o modelo clique [aqui](https://huggingface.co/DaniloTertu/outputs?text=I+like+you.+I+love+you).

# Conclusão

O modelo consegui performar bem tendo em vista que a quantidade de dados é relativamente pequena. Ele teve uma taxa de acerto de 95,2% nos dados de teste. O modelo teve uma grande melhora da primeira época para a segunda, depois disso ele não mudou muito. Após alguns testes no Hugging Face foi possível perceber que o quando o resultado é <0.6 a mensagem não é spam, já quando é >0.6 é spam. Além disso, os valores de não spam ficam muito próximos de 0.5 e os de spam ficam muito próximos de 0.75.